Fine Tuning a T5 transformer for any Summarization Task

In [31]:
import pandas as pd

We start with our custom data that was prepared using the model for summarization on arvix articles. 

Load the pre-trained FLAN T5 model: You can use the AutoModelForSeq2SeqLM class from the Hugging Face Transformers library to load the pre-trained FLAN T5 model. Here is an example:

Tokenize the summary data

In [32]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")


In [33]:
your_text_data= pd.read_csv('/Users/yasminesarraj/Documents/GitHub/M3-Assignment-Deep-Learning/Assignment_4/data/full_texts.csv')

In [34]:
your_text_data.columns

Index(['Unnamed: 0', 'text', 'summary'], dtype='object')

In [35]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(your_text_data, test_size=0.2)

In [38]:
import pandas as pd
from transformers import AutoTokenizer
# load the tokenizer

# read the CSV file using pandas
df = pd.read_csv('/Users/yasminesarraj/Documents/GitHub/M3-Assignment-Deep-Learning/Assignment_4/data/full_texts.csv')
# preprocess and tokenize the text in each row of the CSV file
tokenized_data = []
for _, row in df.iterrows():
    text = row["text"]
    # preprocess the text as needed (e.g., lowercase)
    tokenized_text = tokenizer.encode(text, add_special_tokens=True)
    tokenized_data.append(tokenized_text)

# save the tokenized data in a numpy array
import numpy as np
tokenized_data = np.array(tokenized_data)
np.save("tokenized_data.npy", tokenized_data)

/var/folders/ss/lhx9lhjx3wn1dkd8thzyx0w40000gn/T/ipykernel_66205/1869311398.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tokenized_data = np.array(tokenized_data)


In [41]:
import torch
from torch.utils.data import TensorDataset
# Load the tokenized data (assuming it was saved as a numpy array)
tokenized_data = np.load("tokenized_data.npy", allow_pickle=True)
# Convert the tokenized data to PyTorch tensors
input_ids = torch.tensor([x for x in tokenized_data])
# Create a TensorDataset object
dataset = TensorDataset(input_ids)

ValueError: expected sequence of length 789 at dim 1 (got 1680)

In [27]:
encoded_inputs = tokenizer(tokenized_data, truncation=True, padding=True)


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=your_training_dataset,
    data_collator=your_data_collator,
    tokenizer=tokenizer,
)

trainer.train()
